In [ ]:
!pip install bert_score
!pip install sacrebleu
!pip install --upgrade openpyxl
!pip install jieba

In [2]:
from bert_score import score
import openpyxl
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")

In [3]:
zhcn_df = pd.read_excel("extra_translated_zh-CN.xlsx")

In [4]:
zhcn_df = zhcn_df[["multi_context answer (english translated)", "multi_context answer (zh-CN)"]]
zhcn_df = zhcn_df.rename(columns={
    'multi_context answer (english translated)': 'eng_reference',
    'multi_context answer (zh-CN)': 'chinese_response'
})

In [5]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-zh-en")

zhcn_df['translated_to_eng'] = zhcn_df['chinese_response'].apply(lambda x: translator(x)[0]['translation_text'])

In [6]:
zhcn_df.head()

,eng_reference,chinese_response,translated_to_eng
0,"If you are a victim of fraud, here are some st...",如果您是诈骗的受害者，您可以采取以下一些步骤：\n\n1. 在线或亲自向警方报案。如果情况紧...,"If your account or credit card is stolen, cont..."
1,If you borrow money from an unlicensed moneyle...,如果您从新加坡无牌放贷人那里借钱，您可能会面临严重的处罚。您的工作许可证可能会被吊销，这意味...,If you borrow money from a non-licensed lender...
2,If you receive a call or text message from an ...,如果您接到来自未知号码的电话或短信，声称来自人力部或警察局等政府机构，请务必小心。不要与他们...,If you receive a telephone call or a text mess...
3,You can stay up to date with the latest regula...,您可以通过访问 www.gov.sg 和 www.mom.gov.sg 等官方网站来了解新加...,You can find the latest Singapore legislation ...
4,"If you have financial difficulties, you can se...",如果您有经济困难，可以向社会及家庭发展部社会服务办公室寻求帮助。他们有一条名为 ComCar...,"If you have financial difficulties, you can as..."


In [7]:
def calculate_bertscore_row(row):
    P, R, F1 = score([row['translated_to_eng']], [row['eng_reference']], lang="en")
    return F1.mean().item()

In [8]:
zhcn_df['bert_score_f1'] = zhcn_df.apply(calculate_bertscore_row, axis=1)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

In [9]:
zhcn_df['bert_score_f1'].mean()

0.9396484765139493

In [12]:
# import sacrebleu

# def calculate_bleu_on_translated(row):
#     reference = [row['eng_reference']]
#     hypothesis = row['translated_to_eng']
#     bleu = sacrebleu.sentence_bleu(hypothesis, reference, tokenize="13a")
#     return bleu.score

# zhcn_df['bleu_score_translated'] = zhcn_df.apply(calculate_bleu_on_translated, axis=1)

In [13]:
# zhcn_df['bleu_score_translated'].mean()

28.340005627114365